<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Projeto_Aquecimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXTRAÇÃO DOS DADOS


## Importação das bibliotecas e do conector do mongo

In [ ]:
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient

In [ ]:

uri = "mongodb+srv://caio-soulcode.o09ab0z.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='/content/X509-cert-1702224064282282452.pem')

db = client['proj_aquec']
colecao1 = db['202201_orig']
colecao2 = db['202204_orig']
colecao3 = db['202210_orig']
doc_count = colecao1.count_documents({})
print(doc_count)


152140


In [ ]:
df1orig = pd.read_csv('/content/D.SDA.PDA.005.CAT.202201orig.csv',sep=';',encoding='ISO-8859-1')
df2orig = pd.read_csv('/content/D.SDA.PDA.005.CAT.202204orig.csv',sep=';',encoding='ISO-8859-1')
df3orig = pd.read_csv('/content/D.SDA.PDA.005.CAT.202210orig.csv',sep=';',encoding='ISO-8859-1')

In [ ]:
df1orig_dict = df1orig.to_dict('records')
df2orig_dict = df2orig.to_dict('records')
df3orig_dict = df3orig.to_dict('records')


In [ ]:
df2orig.dtypes

Agente  Causador  Acidente     object
Data Acidente                  object
CBO                            object
CID-10                         object
CNAE2.0 Empregador              int64
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Acidente.1                object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

In [ ]:
# Houve um erro no momento de inserir os dados no mongoDB, foi checada cada coluna de cada DF para achar alguma inconsistência impeditiva 
# Incons.: Indica Óbito Acidente ('{ñ') - a falta do fechamento das chaves pode estar gerando problema na geração dos documentos no mongoDB
# Origem de Cadastramento CAT ('{ñ class') - mesma justif.

sorted(df3orig['Data Nascimento'].unique())

In [179]:
# foi dado um replace em todos os dfs com tais inconsistências
#df1orig.replace(['{','}'],'', regex=True, inplace=True)
df2orig.replace(['{','}'],'', regex=True, inplace=True)
#df3orig.replace(['{','}'],'', regex=True, inplace=True)

In [180]:
# Carregamento para o MongoDB para ser posto em disponibilidade
#colecao1.insert_many(df1orig_dict)
colecao2.insert_many(df2orig_dict)
#colecao3.insert_many(df3orig_dict)

colecao1.count_documents({})

BulkWriteError: ignored

## Enviando o arquivo para o Cloud Storage

In [ ]:
#importação das bibliotecas da cloud storage
!pip install gcsfs

In [ ]:
# Realizando a configuração da conta utilizando a chave json
from google.cloud import storage
import os

serviceAccount = '/content/bold-origin-366512-48dc9e027646.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


In [ ]:
# Acessando o bucket, e inserindo os arquivos
cliente = storage.Client()
bucket = cliente.get_bucket('projeto-aquecimento15-12')
blob = bucket.blob('D.SDA.PDA.005.CAT.202201orig.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202201orig.csv')

In [ ]:
cliente = storage.Client()
bucket = cliente.get_bucket('projeto-aquecimento15-12')
blob = bucket.blob('D.SDA.PDA.005.CAT.202204orig.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202204orig.csv')

In [ ]:
cliente = storage.Client()
bucket = cliente.get_bucket('projeto-aquecimento15-12')
blob = bucket.blob('D.SDA.PDA.005.CAT.202210orig.csv')
blob.upload_from_filename('/content/D.SDA.PDA.005.CAT.202210orig.csv')

# CARREGAMENTO INICIAL DOS DADOS

Agora com os arquivos com disponibilidade em nuvem (GCP e MongoDB), serão carregados para transformação

In [197]:
# Carregando os DBs e os transformando em DFs do Pandas
df = pd.read_csv('https://storage.googleapis.com/projeto-aquecimento15-12/D.SDA.PDA.005.CAT.202201orig.csv',sep=';',encoding = 'ISO-8859-1')
df20 = pd.read_csv('https://storage.googleapis.com/projeto-aquecimento15-12/D.SDA.PDA.005.CAT.202204orig.csv',sep=';',encoding = 'ISO-8859-1')
df30 = pd.read_csv('https://storage.googleapis.com/projeto-aquecimento15-12/D.SDA.PDA.005.CAT.202210orig.csv',sep=';',encoding = 'ISO-8859-1')

In [198]:
# gerando um backup
dfback = df.copy()
df20back = df20.copy()
df30back = df30.copy()

#TRANSFORMAÇÃO COM PANDAS

## Análises primárias do DF1

In [211]:
df.head(2)

,Agente Causador Acidente,CBO,CID-10,CNAE2.0 Empregador,CNAE2.0 Empregador.1,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,...,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Data Acidente.1,Data Despacho Benefício,Data Acidente.2,Data Nascimento,Data Emissão CAT,CNPJ/CEI Empregador
0,{ñ class},515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,8630,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,...,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022,57.571.275.002.570
1,"Motocicleta, Motoneta",519110-Motociclista no Transporte de Document,S52.5 Frat da Extremidade Distal do Radio,4930,Transporte Rodoviario de Carga,Segurado/Dependente,Pa,Empregado,Não,355030-São Paulo,...,Masculino,Trajeto,Maranhão,São Paulo,2022/02,0000/00,25/02/2022,16/12/1990,01/03/2022,19.578.086.000.162


In [214]:
df['Origem de Cadastramento CAT'].unique()

array(['Internet', '{ñ class'], dtype=object)

In [182]:
df.dtypes

Agente  Causador  Acidente     object
Data Acidente                  object
CBO                            object
CID-10                         object
CNAE2.0 Empregador              int64
CNAE2.0 Empregador.1           object
Emitente CAT                   object
Espécie do benefício           object
Filiação Segurado              object
Indica Óbito Acidente          object
Munic Empr                     object
Natureza da Lesão              object
Origem de Cadastramento CAT    object
Parte Corpo Atingida           object
Sexo                           object
Tipo do Acidente               object
UF  Munic.  Acidente           object
UF Munic. Empregador           object
Data Acidente.1                object
Data Despacho Benefício        object
Data Acidente.2                object
Data Nascimento                object
Data Emissão CAT               object
CNPJ/CEI Empregador            object
dtype: object

In [205]:
# Comparando as duas colunas da tabela
df['Data Acidente.1'].equals(df['Data Acidente'])

True

In [209]:
# dropando uma das colunas 
df.drop('Data Acidente',axis=1,inplace=True)

In [ ]:
# Renomeando as colunas para deixá-las mais fáceis
df.rename(columns={'Agente  Causador  Acidente':'ag_causador', 'Data Acidente':'data','CID-10':'cid10','CNAE2.0 Empregador':'cod_cnae',
                   'CNAE2.0 Empregador.1':'desc_cnae','Emitente CAT':'emitente_cat','Espécie do benefício':'tipo_beneficio',
                   'Filiação Segurado':'filiacao_seg','Indica Óbito Acidente':'obito','Munic Empr':'munic_empregador',
                   'Natureza da Lesão':'nat_lesao','':'','':'','':'','':'','':'','':'',})

#TRANSFORMAÇÃO COM PYSPARK

# CARREGAMENTO FINAL